In [ ]:
# define Flink client
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import StreamTableEnvironment
#from pyflink.table import TableEnvironment
#from pyflink.table.expressions import col

env = StreamExecutionEnvironment.get_execution_environment()
env.add_jars("file:///opt/iceberg-flink-runtime.jar")

flink = StreamTableEnvironment.create(env)

flink.execute_sql("""
CREATE CATALOG iceberg WITH (
  'type'='iceberg',
  'catalog-type'='hive',
  'uri'='thrift://hivemetastore:9083',
  'clients'='5',
  'property-version'='1',
  'warehouse'='file://usr/local/hadoop/warehouse'
)
""")

In [1]:
# define Trino client
from trino.dbapi import connect

trino_connection = connect(
    host="trino",
    port=8080,
    user="iceberg",
    #catalog="<catalog>",
    #schema="<schema>",
)
trino = trino_connection.cursor()

In [ ]:
rows = trino.execute("SELECT * FROM system.runtime.nodes").fetchall()
print(rows)

In [2]:
rows = trino.execute("SELECT * FROM iceberg.default.sample").fetchall()
print(rows)

[[2, 'b'], [2, 'b'], [1, 'a'], [1, 'a']]


In [11]:
# define Spark client
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .config("spark.sql.warehouse.dir", "/usr/local/hadoop/warehouse") \
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg.type", "hive") \
    .config("spark.sql.catalog.iceberg.uri", "thrift://hivemetastore:9083") \
    .config("spark.sql.catalog.iceberg.cache-enabled", False) \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/16 19:36:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
spark.sql("""
    DROP TABLE if exists iceberg.default.sample
""")
spark.sql("""
    CREATE TABLE IF NOT EXISTS iceberg.default.sample (
        id bigint,
        data string
    )
    USING iceberg
""")

In [ ]:
spark.sql("""
INSERT INTO iceberg.default.sample VALUES (1, 'a'), (2, 'b')
""")

In [12]:
spark.sql("""
SELECT file_path FROM iceberg.default.sample.all_data_files
""").show()

+--------------------+
|           file_path|
+--------------------+
|file:/usr/local/h...|
|file:/usr/local/h...|
|file:/usr/local/h...|
|file:/usr/local/h...|
+--------------------+



In [25]:
from pyiceberg.catalog.hive import HiveCatalog

catalog = HiveCatalog(name="iceberg", uri="thrift://hivemetastore:9083")
table = catalog.load_table("default.sample")

print(table.history()[0].manifests())

AttributeError: 'SnapshotLogEntry' object has no attribute 'manifests'